In [ ]:
import pandas as pd

In [ ]:
from GlassBox.figs.figs_utils import FIGSGridSearch

FIGS_parameter = dict(
    max_rules= [5,10,15,20,25],
    max_trees=[2,3,4,5,6,7,8,9,10],
    min_impurity_decrease=[0.1, 0.2, 0.3],
)

gs = FIGSGridSearch(path_dataset="../../outputs/scores.csv", random_state=841)
best_ = gs.grid_search(FIGS_parameter)

 99%|█████████▉| 134/135 [06:53<00:07,  7.48s/it, nDCG=0.805]

In [ ]:
best_

In [ ]:
gs.eval_model(model=best_[0], df=gs.test)

In [ ]:
qId = 1

In [ ]:
job_curricula = gs.test[gs.test["qId"] == qId]

In [ ]:
y_pred = best_[0].predict(job_curricula.iloc[:, 2:13])

In [ ]:
y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])

In [ ]:
dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)

In [ ]:
dt_final.sort_values("labels", ascending=False)["labels"].head(15)

In [ ]:
dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

In [ ]:
best_[0].plot()